This is just another lgbm baseline for this competition, a lot of the work, ideas and function are copied from https://www.kaggle.com/tommy1028/lightgbm-starter-with-feature-engineering-idea

The features that improve the model are the aggregartions stats using time_id and stock_id using the realized volatility for different windows from the past

Cheers and have fun

In [ ]:
import math

In [ ]:
#import galearn
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
#from galearn import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score
import pandas as pd
import bqplot
import matplotlib.pyplot as plt
import time
import numpy as np
from tqdm.notebook import tqdm, trange
import seaborn as sns
import optuna
import ipywidgets as widgets
sns.set({'figure.figsize': (12, 8)})
sns.set_color_codes("pastel")
rng = np.random.default_rng()

In [ ]:
#generates a dictionary from the pool of genes
def generate_parent(gene_pool):
    parent = dict()
    for gene in gene_pool.keys():
        parent[gene] = rng.choice(gene_pool[gene])
    fitness = get_fitness(estimator(**parent), fitness_function)
    #print(f"new individuals fitness is {fitness}")
    if math.isnan(fitness):
        print("nan : ", parent)
    return Individual(parent, fitness)

In [ ]:
def mutate(parent, gene_pool):
    gene = rng.choice(list(params))
    child = parent.copy()
    new_gene, alternate = rng.choice(gene_pool[gene], 2)
    #help make sure the gene get's mutated
    child[gene] = alternate if new_gene == child[gene] else new_gene
    return child

In [ ]:
class Individual:
    def __init__(self, genes, fitness):
        self._genes = genes
        self._fitness = fitness
        self._fp = 0 #fitness_proportion to be used when selection == fp
        
    def __eq__(self, other):
        return self.genes == other.genes

    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def __gt__(self, other):
        return self.fitness > other.fitness
    
    def __str__(self):
        return f"Individual with genes: {self._genes} and fitness:{self._fitness}"
        
    @property    
    def genes(self):
        return self._genes
    
    @property
    def fitness(self):
        return self._fitness
    
    #may add cv = cv/skf as an option
    def set_fitness(self):
        self._fitness = get_fitness(estimator(**self.genes), fitness_function)
    
    def set_gene(self, gene, value):
        self._genes[gene] = value
        
    def get_gene_from_window(self, gene):
        min_c = gene_pool[gene].min()
        max_c = gene_pool[gene].max()
        dist_1 = self._genes[gene] - min_c
        dist_2 = max_c - self._genes[gene]
        dist = min(dist_1, dist_2)*gnp_window
        lb = self._genes[gene] - dist
        ub = self._genes[gene] + dist
        new_gene, alternate = rng.choice(gene_pool[gene][(gene_pool[gene] >= lb) & (gene_pool[gene] <= ub)], 2)
        return new_gene, alternate
        
    
    def mutate(self):
        gene = rng.choice(list(self._genes))
        if restrict_gnp and isinstance(gene_pool[gene], float):
            #give chance of diversity = 1-p_mutate until 10% chance
            if rng.random() < p_outlier:
                print(f"got an outlier")
                new_gene, alternate = rng.choice(gene_pool[gene], 2)
            else:
                new_gene, alternate = self.get_gene_from_window(gene)
        else:
            new_gene, alternate = rng.choice(gene_pool[gene], 2)
    #help make sure the gene get's mutated
        self._genes[gene] = alternate if new_gene == self._genes[gene] else new_gene
        return
    
    def get_estimator(self):
        return estimator(**self._genes)

In [ ]:
#creates a population of size size with parameters from gene_pool
def create_population(gene_pool, size = 10):
    population = []
    for i in range(size):
        population.append(generate_parent(gene_pool))
    population.sort(reverse = True)
    return population

In [ ]:
class Population:
    #create initial population
    def __init__(self, gene_pool, size = 10):
        self._population = create_population(gene_pool, size)
        self._size = size
      
    #note that if several individuals have == best fitness anyone of them is returned in the sorted list
    @property
    def best_individual(self):
        return self._population[0]
    
    @property
    def best_fitness(self):
        return self._population[0].fitness
    
    @property
    def population(self):
        return self._population
    
    @property
    def size(self):
        return self._size
    
    def replace_generation(self, new_gen):
        new_gen.sort(reverse = True)
        self._population = new_gen

In [ ]:
def get_fitness(individual, fitness_function, cv = 3):
    score = cross_val_score(individual, X_train, y_train, cv=cv, scoring = fitness_function)
    return score.mean()

In [ ]:
def select_breeding(population, selection = 'truncation', frac = 0.75):
    if selection == 'truncation':
        cut = int(len(population.population)*frac)
        breeding = population.population[:cut]
        return breeding
    elif selection == 'fitness_proportionate' or selection == 'fp':
        size = int(population.size * frac)
        return fp_selection(population, size)
    elif selection == 'tournament':
        size = int(population.size * frac)
        return tournament_selection(population, size)
    elif selection == 'sus':
        size = int(population.size * frac)
        return sus_selection(population, size)

In [ ]:
def fp_selection(pop, size):
    p = np.array([ind.fitness for ind in pop.population])
    total_fitness = p.sum()
    p = p / total_fitness
    #p = np.cumsum(p) nice alternative solution
    return rng.choice(pop.population, size = size, p = p).tolist()

In [ ]:
#stochastic universal sampling
def sus_selection(pop, size):
    p = np.array([ind.fitness for ind in pop.population]).cumsum()
    total_fitness = np.array([ind.fitness for ind in pop.population]).sum()
    step = total_fitness / size
    start = rng.uniform(0, step)
    steps = [(start + i*step) for i in range(size)]
    i = 0
    breeding = []
    for s in steps:
        while p[i] < s:
            i = i + 1
            breeding.append(pop.population[i])
    return breeding

In [ ]:
#add requirement size and elitism have to be even!
#also elitism is almost unnecessary if tournament, almost!
def tournament_selection(pop, size):
    participants = [ind for ind in pop.population]
    breeding = []
    #could implement different rounds here
    #but I think that's almost the same as calling tournament different times with smaller sizes
    for i in range(size):
        a, b = rng.choice(participants, 2)
        if a > b:
            breeding.append(a)
            participants.remove(a)
        else:
            breeding.append(b)
            participants.remove(b)
    return breeding
        
    
#reverse tournament, eliminates need for elitism
#could use with parallelism
def rev_tournament_selection(pop, size):
    breeding = [ind for ind in pop.population]
    num_eliminated = len(breeding) - size
    for i in range(num_eliminated):
        a, b = rng.choice(participants, 2)
        if a > b:
            breeding.remove(b)
        else:
            breeding.remove(a)
    return breeding
        

In [ ]:
def breed(parent_1, parent_2, p_cross, p_mutate):
    # check for recombination
    # if crossover happens at probability p then not crossover would happen at probability 1-p
    #rand() will draw a number larger than p_cross 1-p times
    #and a number < p_cross p times
    # children are copies of parents by default
    child_1, child_2 = Individual(parent_1.genes, parent_1.fitness), Individual(parent_2.genes, parent_2.fitness)
    if np.random.rand() < p_cross: 
        
        genes = list(child_1.genes)
        child_1, child_2 = crossover(parent_1, parent_2, child_1, child_2)
        #mutate if p
    if np.random.rand() < p_mutate:
        child_1.mutate()
    if np.random.rand() < p_mutate:
        child_2.mutate()
        
        child_1.set_fitness()
        child_2.set_fitness()
    return child_1, child_2

In [ ]:
# crossover two parents to create two children
# should not be called by itself because it doesn't set fitness 
def crossover(parent_1, parent_2, child_1, child_2):
    # children are copies of parents by default
    genes = list(child_1.genes) #make global to make more efficient!
    # select crossover point that is not on the end of the string
    start = rng.choice(range(len(genes) - 1))
    #no crossover happening
    if start == len(genes) -1:
        return [child_1, child_2]
    cut = rng.choice(range(start, len(genes)))
    #no crossover happening
    if cut == start:
        return [child_1, child_2]
    # perform crossover
    for gene in genes[start:cut]:
        if isinstance(gene_pool[gene], float): #introduce more diversity by modified crossover for continous values
            #could also solve this with algebra, but I like using the predefined gene_pool
            lower = parent_1[gene]
            higher = parnt_2[gene]
            if parent_1[gene] > parent_2[gene]:
                lower = parent_2[gene]
                higher = parent_1[gene]
                
            new_gene_1, new_gene_2, = rng.choice(gene_pool[gene][(gene_pool[gene] >= lower) & (gene_pool[gene] <= higher)], 2)
            child_1.set_gene(gene, new_gene_1)
            child_2.set_gene(gene, new_gene_2)
        else:
            child_1.set_gene(gene, parent_2.genes[gene])
            child_2.set_gene(gene, parent_1.genes[gene])
        
    return child_1, child_2

In [ ]:
def simulate(params,
             scorer,
             iterations,
             model,
             train_set,
             train_labels,
             selection = 'fp',
             p_cross = 1,
             cv = 3,
             p_mutate = 1,
             sim_ann = True, 
             restrict_gene_pool = True, #narrow genes i.e. finetune
             gene_pool_window = 1.0, #initial size of window
             decay = None,
             elitism = 2,
             population_size = 10):
    #add some fixed genes
    global X_train, y_train, estimator, fitness_function, gene_pool, restrict_gnp, gnp_window, rng
    global p_outlier
    generations = np.arange(0, iterations)
    fitness_prog = []
    p_outlier = 1 - p_mutate
    rng = np.random.default_rng()
    X_train, y_train = train_set, train_labels
    fitness_function = scorer
    estimator = model
    gene_pool = params
    restrict_gnp = restrict_gene_pool
    gnp_window = gene_pool_window
    population = Population(gene_pool, size = population_size)
    best_fitness = population.best_fitness
    if decay == None:
        decay = 1 / iterations
    print(f"best initial fitness: {population.best_fitness}")
    for i in trange(iterations):
        fitness_prog.append(best_fitness)
        new_gen = []
        breeding = select_breeding(population, selection)
        for elite in range(elitism):
            new_gen.append(population.population[elite])
       
        #elitism to be implemented here
        while(len(new_gen) < population.size): #let populatin size oscillate +1 -1?
            parent_1, parent_2 = rng.choice(breeding, 2) #possibility of selecting the same individual
            child_1, child_2 = breed(parent_1, parent_2, p_cross, p_mutate)
            new_gen.append(child_1)
            new_gen.append(child_2)
        #replace the previous generation
        population.replace_generation(new_gen)
        #are you better than the last?
        if (best_fitness < population.best_fitness):
            diff = population.best_fitness - best_fitness
            best_fitness = population.best_fitness
            display(f"child {population.best_individual} with fitness {population.best_fitness}, which is {diff} better than before")
        if sim_ann:
            gnp_window = gnp_window - gnp_window*decay
            p_cross = p_cross - p_cross*decay
            p_mutate = p_mutate - p_mutate*decay
            if p_outlier < 0.1:
                p_outlier = 1 - p_mutate
    #note if several individuals have same fitness anyone of them is returned
    return population.best_individual, (generations, fitness_prog)

In [ ]:
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std]
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)
    
    # Merge all
    df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_450', 'log_return2_realized_volatility_450', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_150', 'log_return2_realized_volatility_150', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_450', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_150']

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return -np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
params = dict()
params['kernel'] = ['rbf', 'poly', 'sigmoid']
params['degree'] = [2, 3, 4]
params['gamma'] = ['scale', 'auto']
params['C'] = np.linspace(0.001, 10, 1000)
params['epsilon'] = np.linspace(0.01, 10, 500)

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
train, test = read_train_test()
x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
y = train['target']
x_test = test.drop(['row_id', 'time_id'], axis = 1)
# Transform stock id to a numeric value
x['stock_id'] = x['stock_id'].astype(int)
x_test['stock_id'] = x_test['stock_id'].astype(int)

In [ ]:
#reg = lgb.LGBMRegressor(**nanpams)

In [ ]:
scorer = make_scorer(rmspe)

In [ ]:
#np.mean(cross_val_score(reg, x, y, scoring=scorer))

In [ ]:
best, history= simulate(params, scorer, 500,
                        SVR,
                        x, y,
                        selection = 'tournament',
                        p_cross = 1,
                        p_mutate = 1,
                        sim_ann = True)

In [ ]:
sns.regplot(x = history[0], y = history[1])
plt.xlabel('generation')
plt.ylabel('fitness')
plt.savefig('svr.png')
plt.show()

In [ ]:
#test_p = reg.predict(x_test)